(encoding-text)=
# Encoding Text    

Consider the following examples of movie reviews:


Example 1 
* Review 1A (negative) <br/>
I thought the movie would be great, *but it is not*. The script is weak, the pacing is slow, and the ending feels pointless.
* Review 1B (positive) <br/>
I thought the movie would not be great, but it *is*. The script is strong, the pacing is brisk, and the ending feels meaningful.


Example 2
* Review 2A (negative)<br/>
The acting is decent, but the plot is predictable and the jokes fall flat.
* Review 2B (positive)<br/>
The plot is predictable and the jokes fall flat, but the acting is decent.


Example 3
* Review 3A (negative)<br/>
For the first ninety minutes I waited for something exciting to happen. Spoiler: it never does.
* Review 3B (positive)<br/>
For the first ninety minutes I waited for something exciting to happen, and when it finally does it is worth every second.


A problem with a **bag of words** representation treats each review as the same multiset of words: 
$$
vector=(acting:1, plot:1, predictable:1, but:1, \ldots).
$$ 

Because position is discarded, the classifier cannot learn that "not" negates what follows or that the clause after "but" usually carries the main sentiment.
 



In [1]:
import numpy as np
from tensorflow.keras.layers import Embedding

# 1. Toy vocabulary: 5 tokens
#    0 = <PAD>, 1 = "good", 2 = "bad", 3 = "not", 4 = "movie"
vocab_size   = 5
embed_dim    = 3   # just three numbers per word
sequence_len = 4

embed = Embedding(input_dim=vocab_size,
                  output_dim=embed_dim,
                  input_length=sequence_len)

# 2. Example review, integer‑encoded and padded:
#    "not good movie"   →   [3, 1, 4, 0]
sample = np.array([[3, 1, 4, 0]])        # shape (1, 4)

dense_seq = embed(sample)                # shape (1, 4, 3)
print(dense_seq.numpy().round(2))


[[[-0.03 -0.01 -0.  ]
  [ 0.03  0.02 -0.01]
  [-0.    0.01 -0.01]
  [-0.02  0.01  0.03]]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-05-06 06:33:07.117041: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2025-05-06 06:33:07.117062: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 96.00 GB
2025-05-06 06:33:07.117066: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 36.00 GB
I0000 00:00:1746527587.117077  658944 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1746527587.117098  658944 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Word Embeddings

Unlike bag‑of‑words, use dense vectors that are trainable and will gradually move so that “good” and “bad” point in different directions, while “not” can flip the meaning when a sequential layer (CNN, RNN) reads the tokens in order.



In [2]:
import numpy as np
from tensorflow.keras.layers import Embedding

# 1. Toy vocabulary: 5 tokens
#    0 = <PAD>, 1 = "good", 2 = "bad", 3 = "not", 4 = "movie"
vocab_size   = 6
embed_dim    = 2   # just three numbers per word
sequence_len = 4

embed = Embedding(input_dim=vocab_size,
                  output_dim=embed_dim,
                  input_length=sequence_len)

# 2. Example review, integer‑encoded and padded:
#    "not good movie"   →   [3, 1, 4, 0]
sample = np.array([[3, 1, 5, 3]])        # shape (1, 4)

dense_seq = embed(sample)                # shape (1, 4, 3)
print(dense_seq.numpy().round(2))


[[[-0.03  0.05]
  [ 0.02 -0.01]
  [ 0.03  0.02]
  [-0.03  0.05]]]


In [3]:
import tensorflow as tf


## Using a CNN

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ------------------------------------------------------------------
# 1. Data loading and preprocessing
# ------------------------------------------------------------------
max_features = 20_000      # keep the 20 000 most frequent words
maxlen        = 400        # cut / pad every review to 400 tokens

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    num_words=max_features
)

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = keras.preprocessing.sequence.pad_sequences(x_test,  maxlen=maxlen)

# ------------------------------------------------------------------
# 2. CNN model
# ------------------------------------------------------------------
model = keras.Sequential([
    layers.Embedding(max_features, 128, input_length=maxlen),

    layers.Conv1D(64, 7, activation="relu"),
    layers.MaxPooling1D(3),

    layers.Conv1D(64, 7, activation="relu"),
    layers.GlobalMaxPooling1D(),

    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ------------------------------------------------------------------
# 3. Training
# ------------------------------------------------------------------
history = model.fit(
    x_train,
    y_train,
    epochs=8,
    batch_size=128,
    validation_split=0.2,
    verbose=0
)

# ------------------------------------------------------------------
# 4. Evaluation
# ------------------------------------------------------------------
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

# ------------------------------------------------------------------
# 5. Predictions (first ten test examples, rounded for readability)
# ------------------------------------------------------------------
preds = model.predict(x_test[:10]).round(3).squeeze()
print("Predicted probabilities:", preds)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

2025-05-06 06:33:09.491866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

A CNN is great at spotting local phrases like “not good,” but it quickly forgets where it found them; sentiment often hinges on relations far apart in the text, so we need a model that can keep track of information across the whole sequence.

* A Conv1D with kernel size $k=7$ can only look at $7$ consecutive tokens at a time.
* Stacking layers enlarges the receptive field only linearly (first layer sees 7 tokens, two layers see $\approx 19$, etc.).
* Important cues in a review are often dozens of tokens apart ("I hoped it would be great...but it is not.").

On the IMDB dataset, a  CNN typically reaches $~0.88$ accuracy, identical to the bag‑of‑words MLP, then plateaus. Extra filters or layers add parameters but do not solve the fundamental distance problem.

